In [1]:
import geemap, ee
import math

# ee.Authenticate()
# ee.Initialize()
Map = geemap.Map()
# Map

In [2]:
# roi01 = geemap.shp_to_ee(r"D:\\work\\gee_\\gee_code\\土壤湿度反演\\region\\area.shp").geometry()
roi01 = ee.FeatureCollection(r'projects/ee-lmike18770616551/assets/area').geometry()
r = roi = ee.Geometry.Rectangle([70, 0, 138, 55], None, False)

In [3]:

# Map.centerObject(roi01,7)

# 加载光学、SAR数据 
# 筛选光学影像并计算 MNDWI 
Optical_dataset = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")

dataset1 = Optical_dataset\
                      .filterDate("2023-1-1","2023-12-31")\
                      .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',5))
image1=dataset1.median().divide(10000)
Map.addLayer(image1, {'min': 0, 'max': 0.3,'bands': ['B4', 'B3', 'B2']},"RGB")

visParam = {  
  'min': -1,   
  'max': 1,  
  'palette': ["FFFFFF", "CE7E45", "DF923D", "F1B555", "FCD163",   
            "99B718", "74A901", "66A000", "529400", "3E8601",   
            "207401", "056201", "004C00", "023B01", "012E01",   
            "011D01", "011301"]  
}  

MNDWI01=image1.expression("(R-SWIR)/(R+SWIR)",{
                  "R":image1.select("B4"),"SWIR":image1.select("B11")}).rename('MNDWI')
# print(MNDWI01)
Map.addLayer(MNDWI01, visParam, "MNDWI01")
# 筛选SAR影像并计算，找到一副与光学影像日期靠近的影像 
SARdataset = ee.ImageCollection('COPERNICUS/S1_GRD')\
        .filterDate("2023-1-1","2023-12-31")\
        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))\
        .filter(ee.Filter.eq('instrumentMode', 'IW'))
# imgVV01=ee.Image('COPERNICUS/S1_GRD/S1A_IW_GRDH_1SDV_20230808T103530_20230808T103555_049783_05FC90_EE91')
imgVV01 = SARdataset.median()
# 对用于水云模型的影像进行波段叠加
sar_vv= imgVV01.select('VV').rename('all_ref')             
sar_angle=imgVV01.select('angle') 
# Map.addLayer(imgVV01.select('VV'), {min: -25, max: 5})
Map.addLayer(sar_vv, {min: -25, max: 5},'VV')
# Map.addLayer(sar_angle, {min: 0, max: 90})
all_layer = sar_vv.addBands(sar_angle).addBands(MNDWI01)

# 参照水云模型的公式对土壤后向散射系数求解
def soil_ref(image):
  # 计算植被层的双程衰减因子
  T2 = image.expression(
    'exp(-2 * 0.3334 * MNDWI / cos(angle))', {
      'MNDWI': image.select('MNDWI'),
      'angle': image.select('angle')})
  # 植被层后向反射系数
  veg_back = image.expression(
    '0.1449 * MNDWI * cos(angle) * (1 - T2)', {
      'MNDWI': image.select('MNDWI'),
      'angle': image.select('angle'),
      'T2': T2})
  # 计算土壤层后向反射系数
  soil_back = image.select('all_ref').subtract(veg_back).divide(T2)
  # 返回 soil_back
  return soil_back
# 应用 soil_ref 函数
soil = soil_ref(all_layer)

soil = soil.multiply(soil.gt(-50)).multiply(soil.lt(0))

# 导出数据 
# Export.image.toDrive({
#   image: soil,
#   description: 'soil_ref',
#   scale: 10,
#   region: roi01,
#   maxPixels: 1e13
# })

In [4]:
soil = soil.clip(roi01).rename('soil')

In [5]:
# imgVV01=ee.Image('COPERNICUS/S1_GRD/S1A_IW_GRDH_1SDV_20230808T103530_20230808T103555_049783_05FC90_EE91')
# geemap.download_ee_image(soil.clip(roi01),r'D:\work\gee_\gee_download\土壤湿度反演\soil.tif',region=roi01,scale=10,crs='EPSG:4326')
# geemap.download_ee_image(SM.clip(roi01),r'D:\work\gee_\gee_download\土壤湿度反演\SM.tif',region=roi01,scale=10,crs='EPSG:4326')
# randomPoints = ee.FeatureCollection.randomPoints(region= roi01,points= 500)

In [6]:

SM = ee.Image('projects/ee-lmike18770616551/assets/soil_moisture').rename('SM')

# randomPoints = ee.FeatureCollection.randomPoints(region= roi01,points= 500)

samplePoints = SM.sample(region=roi01,numPixels=500,geometries=True)

trainingData = soil.sampleRegions(collection=samplePoints,
  properties=['SM'],  
  scale=10
)

rf = ee.Classifier.smileRandomForest(100)\
  .setOutputMode('REGRESSION')\
  .train(
    features=trainingData,
    classProperty='SM',  
    inputProperties=['soil']  
  )

soil_moisture = soil.classify(rf)

In [9]:
geemap.download_ee_image(soil_moisture.clip(roi01),r'D:\work\gee_\gee_download\土壤湿度反演\SM_30.tif',region=roi01,scale=30,crs='EPSG:4527',max_tile_dim=1000)
# geemap.ee_export_image_to_drive(soil_moisture.clip(roi01),description = 'soil_moisture',scale = 10,maxPixels = 1e13)

SM_30.tif: |          | 0.00/15.5M (raw) [  0.0%] in 00:00 (eta:     ?)